Part A

In [1]:
# spark initialisation, needs to be done before later imports
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession

from dateutil.parser import parse
from datetime import datetime


datafile='weather/*.csv'  #for all files
datafile='weather/SF04.csv'  #to test on just one file

### can load data using SparkSession like this but then will have to explicitly handle format conversions
#spark=SparkSession.builder.getOrCreate()
#df=spark.read.csv(datafile,header=True)

### better to load data using SQLContext and csv schema (which deals with formatting)
conf = SparkConf().setAppName("wind-sfpd")
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)


df = sqlc.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(datafile)



def date_and_hour(s):
    dt = parse(s.replace('?',' '))
    hour = dt.hour
    return (dt.strftime("%Y-%m-%d"), hour)



In [2]:
df.show()

+----------+-------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|Station_ID|       Station_Name|Location_Label|Interval_Minutes|Interval_End_Time|Wind_Velocity_Mtr_Sec|Wind_Direction_Variance_Deg|Wind_Direction_Deg|Ambient_Temperature_Deg_C|Global_Horizontal_Irradiance|
+----------+-------------------+--------------+----------------+-----------------+---------------------+---------------------------+------------------+-------------------------+----------------------------+
|      SF04|Lincoln High School| 2162 24th Ave|               5| 2014-01-4? 08:15|                 null|                      14.18|             26.31|                    12.81|                       0.005|
|      SF04|Lincoln High School| 2162 24th Ave|               5| 2014-01-4? 08:20|                1.491|                      12.13|             25.96|                    1

In [3]:
stations=df.rdd

In [4]:
#just get the fields of interest and parse the date and hour from the Interval_End_Time
tidied = stations.map(lambda r: (r.Station_ID, date_and_hour(r.Interval_End_Time)[0],date_and_hour(r.Interval_End_Time)[1],r.Ambient_Temperature_Deg_C, r.Wind_Velocity_Mtr_Sec))
#(s,d,h,t,w)
#filter out zero and null values
filtered = tidied.filter( lambda r:  not (r[3]==0.0 and r[4]==0.0))

filter2 = filtered.filter(lambda r:  bool(r[3]) and  bool(r[4]))

#group first three columns as key and last two columns as value
#add a column of 1s which will be used for counting
mapped = filter2.map( lambda r: ((r[0],r[1],r[2]),(r[3],r[4],1)))

#group by key and sum all of the values (t,w,c)
reduced = mapped.reduceByKey( lambda a, b: (a[0]+b[0],a[1]+b[1],a[2]+b[2]))

#calculate averages
averaged = reduced.map(lambda x: (x[0], (x[1][0]/x[1][2], x[1][1]/x[1][2])))

#output results
for (key,value) in averaged.sortByKey().collect():
	print(key,value)



('SF04', '2014-01-01', 8) (12.085, 2.236375)
('SF04', '2014-01-01', 9) (13.097, 1.5883000000000003)
('SF04', '2014-01-01', 10) (14.726041666666662, 2.1740833333333334)
('SF04', '2014-01-01', 11) (16.376458333333332, 2.1182083333333335)
('SF04', '2014-01-01', 12) (16.627708333333334, 2.084979166666667)
('SF04', '2014-01-01', 13) (17.40083333333334, 1.7878958333333337)
('SF04', '2014-01-01', 14) (17.80083333333333, 1.5741249999999998)
('SF04', '2014-01-01', 15) (16.636458333333334, 1.7833750000000002)
('SF04', '2014-01-01', 16) (15.145625, 1.6852708333333328)
('SF04', '2014-01-01', 17) (14.621842105263156, 1.765552631578947)
('SF04', '2014-01-01', 18) (16.123750000000005, 1.7480416666666667)
('SF04', '2014-01-01', 19) (15.704583333333337, 1.401625)
('SF04', '2014-01-01', 20) (14.978750000000003, 0.7526666666666668)
('SF04', '2014-01-01', 21) (13.95875, 1.5961666666666663)
('SF04', '2014-01-01', 22) (14.046470588235294, 0.6316470588235295)
('SF04', '2014-01-01', 23) (14.12909090909091, 0.

('SF04', '2014-11-07', 14) (17.932586206896552, 3.0602241379310344)
('SF04', '2014-11-07', 15) (17.35631578947368, 3.0101754385964905)
('SF04', '2014-11-07', 16) (17.833437500000006, 2.9713125)
('SF04', '2014-12-01', 7) (7.283, 1.194)
('SF04', '2014-12-01', 8) (8.630153846153846, 6.03176923076923)
('SF04', '2014-12-01', 9) (10.869285714285718, 1.4454642857142854)
('SF04', '2014-12-01', 10) (13.254523809523809, 1.3695714285714289)
('SF04', '2014-12-01', 11) (13.598717948717946, 1.415666666666667)
('SF04', '2014-12-01', 12) (14.191627906976745, 1.5642558139534881)
('SF04', '2014-12-01', 13) (14.032, 2.0406857142857135)
('SF04', '2014-12-01', 14) (14.310909090909094, 1.9333636363636364)
('SF04', '2014-12-01', 15) (14.31466666666667, 1.7107333333333332)
('SF04', '2014-12-01', 16) (14.614166666666668, 1.33875)
('SF04', '2014-12-02', 8) (12.923200000000003, 1.06005)
('SF04', '2014-12-02', 9) (14.257222222222225, 0.9937777777777776)
('SF04', '2014-12-02', 10) (15.056304347826087, 1.1038695652

## Part B

In [6]:
from dateutil.parser import parse
from datetime import datetime
from scipy import spatial
from numpy import array

latlongs=[[37.7816834,-122.3887657],\
[37.7469112,-122.4821759],\
[37.7411022,-120.804151],\
[37.4834543,-122.3187302],\
[37.7576436,-122.3916382],\
[37.7970013,-122.4140409],\
[37.748496,-122.4567461],\
[37.7288155,-122.4210133],\
[37.5839487,-121.9499339],\
[37.7157156,-122.4145311],\
[37.7329613,-122.5051491],\
[37.7575891,-122.3923824],\
[37.7521169,-122.4497687]]

locations = ["SF18", "SF04", "SF15", "SF17", "SF36", "SF37", "SF07", "SF11", "SF12", "SF14", "SF16", "SF19", "SF34"]

def locate(l,index,locations):
	distance,i = index.query(l)
	return locations[i]

#locate([37.7736224122729,-122.463749926391],spatial.KDTree(array(latlongs)),locations)




In [51]:
incidentdata="incidents/sfpd.csv"
#for testing
#incidentdata="incidents/sfpd1000.csv"
sfpd = sqlc.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(incidentdata)



In [52]:
sfpd.show()

+----------+--------------------+--------------------+---------+----------+-----+----------+--------------+--------------------+-----------------+----------------+--------------------+--------------+
|IncidntNum|            Category|            Descript|DayOfWeek|      Date| Time|PdDistrict|    Resolution|             Address|                X|               Y|            Location|          PdId|
+----------+--------------------+--------------------+---------+----------+-----+----------+--------------+--------------------+-----------------+----------------+--------------------+--------------+
| 150944099|        NON-CRIMINAL|AIDED CASE, MENTA...|Wednesday|10/28/2015|23:46|  NORTHERN|          NONE|JACKSON ST / VANN...|-122.423049294192|37.7939338236045|(37.7939338236045...|15094409964020|
| 150944005|      OTHER OFFENSES|DRIVERS LICENSE, ...|Wednesday|10/28/2015|23:42|   MISSION|ARREST, BOOKED|   CAPP ST / 19TH ST| -122.41812008649|37.7603012340397|(37.7603012340397...|15094400565016|


In [53]:

#pull out date, time and lat longs, filter for 2014 dates
fixed = sfpd.rdd.map(lambda r: (parse(r.Date),parse(r.Time).hour, [r.Y,r.X]))

only2014 = fixed.filter(lambda r: r[0].year == 2014)



In [39]:
#for k,v,i in only2014.collect():
#    print(k,v,i)

2015-10-28 00:00:00 23 [37.7939338236045, -122.423049294192]
2015-10-28 00:00:00 23 [37.7603012340397, -122.41812008649]
2015-10-28 00:00:00 23 [37.775420706711, -122.403404791479]
2015-10-28 00:00:00 23 [37.7541123443338, -122.399479292854]
2015-10-28 00:00:00 23 [37.7817492235226, -122.396381836885]
2015-10-28 00:00:00 23 [37.7391460959559, -122.416820868342]
2015-10-28 00:00:00 23 [37.7946047720283, -122.410307421926]
2015-10-28 00:00:00 23 [37.7837358666831, -122.488217575328]
2015-10-28 00:00:00 23 [37.7736224122729, -122.463749926391]
2015-10-28 00:00:00 22 [37.7336324732528, -122.382280587597]
2015-10-28 00:00:00 22 [37.7336324732528, -122.382280587597]
2015-10-28 00:00:00 22 [37.7336324732528, -122.382280587597]
2015-10-28 00:00:00 22 [37.7938511491584, -122.409543626564]
2015-10-28 00:00:00 22 [37.7769313104957, -122.426360535365]
2015-10-28 00:00:00 22 [37.7882799630475, -122.423598600854]
2015-10-28 00:00:00 22 [37.7804777480508, -122.412515341356]
2015-10-28 00:00:00 22 [37

2015-10-26 00:00:00 23 [37.7182363858656, -122.455111164447]
2015-10-26 00:00:00 23 [37.7617597243598, -122.435150099812]
2015-10-26 00:00:00 23 [37.7762310404758, -122.414714295579]
2015-10-26 00:00:00 23 [37.7602340753605, -122.419208249507]
2015-10-26 00:00:00 23 [37.7617597243598, -122.435150099812]
2015-10-26 00:00:00 23 [37.7921222370469, -122.423544081104]
2015-10-26 00:00:00 22 [37.773099482048, -122.415564465695]
2015-10-26 00:00:00 22 [37.773099482048, -122.415564465695]
2015-10-26 00:00:00 22 [37.773099482048, -122.415564465695]
2015-10-26 00:00:00 22 [37.7842973843312, -122.410157073911]
2015-10-26 00:00:00 22 [37.724912078558, -122.475003009213]
2015-10-26 00:00:00 22 [37.7310650252983, -122.474855245968]
2015-10-26 00:00:00 22 [37.7191834462785, -122.386437671186]
2015-10-26 00:00:00 22 [37.7881906054546, -122.418580508889]
2015-10-26 00:00:00 22 [37.7990945041537, -122.412285208626]
2015-10-26 00:00:00 22 [37.7535351242762, -122.397201937993]
2015-10-26 00:00:00 22 [37.7

In [54]:
remapped = only2014.map(lambda r: (r[0].strftime("%Y-%m-%d"), r[1], r[2]))


# map latlongs to postcodes and count
#r=(d,h,l)
located = remapped.map(lambda r: (locate(r[2], spatial.KDTree(array(latlongs)),locations ),r[0],r[1]))

#r=(l,d,h)
counted = located.map(lambda r: (r,1))
reduced = counted.reduceByKey(lambda a,b: a+b)

#display
for key,value in reduced.collect():
    print(key,value)



('SF37', '2014-12-31', 23) 18
('SF07', '2014-12-31', 22) 1
('SF14', '2014-12-31', 21) 2
('SF11', '2014-12-31', 21) 6
('SF19', '2014-12-31', 20) 5
('SF34', '2014-12-31', 20) 2
('SF18', '2014-12-31', 20) 2
('SF36', '2014-12-31', 20) 1
('SF37', '2014-12-31', 19) 11
('SF07', '2014-12-31', 18) 1
('SF14', '2014-12-31', 17) 1
('SF19', '2014-12-31', 16) 2
('SF18', '2014-12-31', 16) 1
('SF34', '2014-12-31', 16) 2
('SF04', '2014-12-31', 15) 1
('SF37', '2014-12-31', 15) 7
('SF16', '2014-12-31', 15) 1
('SF19', '2014-12-31', 12) 1
('SF34', '2014-12-31', 12) 4
('SF18', '2014-12-31', 12) 1
('SF37', '2014-12-31', 11) 4
('SF04', '2014-12-31', 11) 3
('SF11', '2014-12-31', 9) 1
('SF19', '2014-12-31', 8) 1
('SF34', '2014-12-31', 8) 1
('SF37', '2014-12-31', 7) 2
('SF19', '2014-12-31', 4) 2
('SF04', '2014-12-31', 3) 3
('SF37', '2014-12-31', 3) 1
('SF07', '2014-12-31', 2) 1
('SF19', '2014-12-31', 0) 3
('SF34', '2014-12-31', 0) 2
('SF18', '2014-12-31', 0) 3
('SF36', '2014-12-31', 0) 1
('SF11', '2014-12-30', 2

('SF34', '2014-11-27', 3) 4
('SF18', '2014-11-27', 3) 1
('SF14', '2014-11-27', 2) 1
('SF11', '2014-11-27', 2) 1
('SF07', '2014-11-27', 1) 1
('SF37', '2014-11-27', 0) 3
('SF04', '2014-11-27', 0) 1
('SF18', '2014-11-26', 22) 5
('SF19', '2014-11-26', 22) 2
('SF34', '2014-11-26', 22) 1
('SF37', '2014-11-26', 21) 4
('SF04', '2014-11-26', 21) 4
('SF14', '2014-11-26', 19) 1
('SF11', '2014-11-26', 19) 1
('SF18', '2014-11-26', 18) 4
('SF36', '2014-11-26', 18) 3
('SF19', '2014-11-26', 18) 2
('SF34', '2014-11-26', 18) 3
('SF04', '2014-11-26', 17) 2
('SF37', '2014-11-26', 17) 11
('SF11', '2014-11-26', 15) 1
('SF18', '2014-11-26', 14) 5
('SF36', '2014-11-26', 14) 1
('SF19', '2014-11-26', 14) 1
('SF37', '2014-11-26', 13) 4
('SF04', '2014-11-26', 13) 4
('SF07', '2014-11-26', 12) 1
('SF14', '2014-11-26', 11) 2
('SF11', '2014-11-26', 11) 1
('SF34', '2014-11-26', 10) 2
('SF36', '2014-11-26', 10) 1
('SF18', '2014-11-26', 10) 1
('SF37', '2014-11-26', 9) 9
('SF18', '2014-11-26', 6) 1
('SF37', '2014-11-26',

('SF34', '2014-10-30', 18) 4
('SF19', '2014-10-30', 18) 2
('SF18', '2014-10-30', 18) 4
('SF36', '2014-10-30', 18) 3
('SF37', '2014-10-30', 17) 14
('SF36', '2014-10-30', 14) 1
('SF19', '2014-10-30', 14) 2
('SF18', '2014-10-30', 14) 1
('SF37', '2014-10-30', 13) 9
('SF04', '2014-10-30', 13) 4
('SF07', '2014-10-30', 12) 1
('SF18', '2014-10-30', 10) 1
('SF37', '2014-10-30', 9) 5
('SF18', '2014-10-30', 6) 1
('SF36', '2014-10-30', 6) 1
('SF19', '2014-10-30', 2) 2
('SF34', '2014-10-30', 2) 2
('SF18', '2014-10-30', 2) 4
('SF37', '2014-10-30', 1) 8
('SF07', '2014-10-30', 0) 1
('SF19', '2014-10-29', 23) 12
('SF18', '2014-10-29', 23) 7
('SF34', '2014-10-29', 23) 1
('SF11', '2014-10-29', 22) 5
('SF07', '2014-10-29', 21) 2
('SF37', '2014-10-29', 20) 16
('SF04', '2014-10-29', 20) 3
('SF16', '2014-10-29', 20) 1
('SF19', '2014-10-29', 19) 5
('SF18', '2014-10-29', 19) 2
('SF36', '2014-10-29', 19) 1
('SF14', '2014-10-29', 18) 2
('SF11', '2014-10-29', 18) 10
('SF37', '2014-10-29', 16) 12
('SF19', '2014-10

('SF07', '2014-09-30', 2) 1
('SF11', '2014-09-30', 1) 1
('SF34', '2014-09-30', 0) 8
('SF19', '2014-09-30', 0) 2
('SF37', '2014-09-29', 23) 5
('SF11', '2014-09-29', 21) 4
('SF14', '2014-09-29', 21) 1
('SF36', '2014-09-29', 20) 4
('SF19', '2014-09-29', 20) 2
('SF18', '2014-09-29', 20) 4
('SF37', '2014-09-29', 19) 14
('SF07', '2014-09-29', 18) 3
('SF14', '2014-09-29', 17) 1
('SF18', '2014-09-29', 16) 5
('SF34', '2014-09-29', 16) 5
('SF37', '2014-09-29', 15) 12
('SF04', '2014-09-29', 15) 1
('SF07', '2014-09-29', 14) 1
('SF11', '2014-09-29', 13) 1
('SF18', '2014-09-29', 12) 1
('SF19', '2014-09-29', 12) 3
('SF04', '2014-09-29', 11) 5
('SF37', '2014-09-29', 11) 9
('SF34', '2014-09-29', 8) 5
('SF18', '2014-09-29', 8) 1
('SF19', '2014-09-29', 8) 2
('SF37', '2014-09-29', 7) 5
('SF34', '2014-09-29', 4) 1
('SF18', '2014-09-29', 4) 1
('SF37', '2014-09-29', 3) 2
('SF07', '2014-09-29', 2) 1
('SF11', '2014-09-29', 1) 1
('SF19', '2014-09-29', 0) 1
('SF18', '2014-09-29', 0) 5
('SF37', '2014-09-28', 22) 

('SF37', '2014-08-28', 18) 9
('SF04', '2014-08-28', 18) 4
('SF18', '2014-08-28', 17) 4
('SF34', '2014-08-28', 17) 2
('SF19', '2014-08-28', 17) 1
('SF11', '2014-08-28', 16) 1
('SF07', '2014-08-28', 15) 1
('SF37', '2014-08-28', 14) 10
('SF04', '2014-08-28', 14) 1
('SF36', '2014-08-28', 13) 2
('SF18', '2014-08-28', 13) 2
('SF19', '2014-08-28', 13) 1
('SF34', '2014-08-28', 13) 1
('SF14', '2014-08-28', 12) 3
('SF11', '2014-08-28', 12) 2
('SF07', '2014-08-28', 11) 1
('SF37', '2014-08-28', 10) 5
('SF16', '2014-08-28', 10) 1
('SF18', '2014-08-28', 9) 1
('SF19', '2014-08-28', 9) 4
('SF37', '2014-08-28', 6) 2
('SF18', '2014-08-28', 5) 1
('SF11', '2014-08-28', 4) 1
('SF37', '2014-08-28', 2) 3
('SF18', '2014-08-28', 1) 4
('SF19', '2014-08-28', 1) 2
('SF34', '2014-08-28', 1) 1
('SF11', '2014-08-28', 0) 1
('SF14', '2014-08-28', 0) 1
('SF37', '2014-08-27', 23) 8
('SF14', '2014-08-27', 21) 2
('SF19', '2014-08-27', 20) 2
('SF34', '2014-08-27', 20) 3
('SF36', '2014-08-27', 20) 1
('SF37', '2014-08-27', 1

...

('SF19', '2014-05-07', 18) 3
('SF36', '2014-05-07', 18) 1
('SF18', '2014-05-07', 18) 2
('SF37', '2014-05-07', 17) 10
('SF04', '2014-05-07', 17) 2
('SF07', '2014-05-07', 16) 2
('SF11', '2014-05-07', 15) 2
('SF14', '2014-05-07', 15) 1
('SF19', '2014-05-07', 14) 2
('SF18', '2014-05-07', 14) 1
('SF34', '2014-05-07', 14) 1
('SF37', '2014-05-07', 13) 5
('SF04', '2014-05-07', 13) 1
('SF14', '2014-05-07', 11) 1
('SF36', '2014-05-07', 10) 1
('SF18', '2014-05-07', 10) 1
('SF34', '2014-05-07', 10) 1
('SF37', '2014-05-07', 9) 10
('SF14', '2014-05-07', 7) 3
('SF18', '2014-05-07', 6) 2
('SF19', '2014-05-07', 6) 1
('SF37', '2014-05-07', 5) 4
('SF14', '2014-05-07', 3) 1
('SF18', '2014-05-07', 2) 1
('SF19', '2014-05-07', 2) 1
('SF37', '2014-05-07', 1) 3
('SF37', '2014-05-06', 23) 8
('SF04', '2014-05-06', 23) 4
('SF07', '2014-05-06', 22) 1
('SF11', '2014-05-06', 21) 2
('SF34', '2014-05-06', 20) 3
('SF18', '2014-05-06', 20) 2
('SF36', '2014-05-06', 20) 1
('SF37', '2014-05-06', 19) 13
('SF04', '2014-05-06

('SF34', '2014-04-05', 21) 3
('SF36', '2014-04-05', 21) 1
('SF19', '2014-04-05', 21) 1
('SF14', '2014-04-05', 20) 6
('SF11', '2014-04-05', 20) 3
('SF07', '2014-04-05', 19) 1
('SF37', '2014-04-05', 18) 14
('SF34', '2014-04-05', 17) 2
('SF18', '2014-04-05', 17) 2
('SF19', '2014-04-05', 17) 1
('SF11', '2014-04-05', 16) 1
('SF14', '2014-04-05', 16) 1
('SF37', '2014-04-05', 14) 9
('SF19', '2014-04-05', 13) 7
('SF18', '2014-04-05', 13) 1
('SF34', '2014-04-05', 13) 1
('SF11', '2014-04-05', 8) 2
('SF07', '2014-04-05', 7) 2
('SF37', '2014-04-05', 6) 5
('SF34', '2014-04-05', 5) 1
('SF11', '2014-04-05', 4) 2
('SF37', '2014-04-05', 2) 8
('SF34', '2014-04-05', 1) 2
('SF18', '2014-04-05', 1) 3
('SF19', '2014-04-05', 1) 1
('SF11', '2014-04-05', 0) 3
('SF14', '2014-04-05', 0) 1
('SF07', '2014-04-04', 23) 1
('SF37', '2014-04-04', 22) 5
('SF19', '2014-04-04', 21) 2
('SF18', '2014-04-04', 21) 3
('SF36', '2014-04-04', 21) 1
('SF34', '2014-04-04', 21) 6
('SF11', '2014-04-04', 20) 5
('SF07', '2014-04-04', 1

('SF04', '2014-03-04', 13) 2
('SF11', '2014-03-04', 11) 1
('SF18', '2014-03-04', 10) 2
('SF37', '2014-03-04', 9) 8
('SF16', '2014-03-04', 9) 1
('SF07', '2014-03-04', 8) 1
('SF11', '2014-03-04', 7) 2
('SF18', '2014-03-04', 6) 2
('SF34', '2014-03-04', 6) 3
('SF37', '2014-03-04', 5) 2
('SF37', '2014-03-04', 1) 4
('SF04', '2014-03-04', 1) 2
('SF18', '2014-03-03', 23) 1
('SF19', '2014-03-03', 23) 2
('SF36', '2014-03-03', 23) 1
('SF11', '2014-03-03', 22) 2
('SF37', '2014-03-03', 20) 5
('SF04', '2014-03-03', 20) 2
('SF18', '2014-03-03', 19) 3
('SF34', '2014-03-03', 19) 3
('SF19', '2014-03-03', 19) 2
('SF11', '2014-03-03', 18) 5
('SF14', '2014-03-03', 18) 1
('SF37', '2014-03-03', 16) 12
('SF04', '2014-03-03', 16) 2
('SF34', '2014-03-03', 15) 3
('SF19', '2014-03-03', 15) 2
('SF18', '2014-03-03', 15) 1
('SF11', '2014-03-03', 14) 2
('SF14', '2014-03-03', 14) 1
('SF37', '2014-03-03', 12) 6
('SF18', '2014-03-03', 11) 3
('SF19', '2014-03-03', 11) 2
('SF34', '2014-03-03', 11) 1
('SF14', '2014-03-03',

('SF19', '2014-01-31', 17) 2
('SF36', '2014-01-31', 17) 1
('SF34', '2014-01-31', 17) 1
('SF14', '2014-01-31', 16) 1
('SF37', '2014-01-31', 14) 8
('SF36', '2014-01-31', 13) 1
('SF19', '2014-01-31', 13) 2
('SF11', '2014-01-31', 12) 1
('SF07', '2014-01-31', 11) 1
('SF16', '2014-01-31', 10) 1
('SF37', '2014-01-31', 10) 2
('SF04', '2014-01-31', 10) 6
('SF34', '2014-01-31', 9) 2
('SF36', '2014-01-31', 9) 1
('SF18', '2014-01-31', 9) 2
('SF37', '2014-01-31', 6) 4
('SF19', '2014-01-31', 5) 1
('SF37', '2014-01-31', 2) 5
('SF18', '2014-01-31', 1) 2
('SF34', '2014-01-31', 1) 1
('SF11', '2014-01-31', 0) 3
('SF14', '2014-01-31', 0) 1
('SF14', '2014-01-30', 23) 2
('SF11', '2014-01-30', 23) 4
('SF19', '2014-01-30', 22) 1
('SF37', '2014-01-30', 21) 7
('SF14', '2014-01-30', 19) 5
('SF11', '2014-01-30', 19) 1
('SF34', '2014-01-30', 18) 2
('SF37', '2014-01-30', 17) 12
('SF16', '2014-01-30', 17) 1
('SF07', '2014-01-30', 16) 1
('SF14', '2014-01-30', 15) 4
('SF11', '2014-01-30', 15) 1
('SF34', '2014-01-30', 

('SF19', '2014-01-01', 1) 3
('SF34', '2014-01-01', 1) 2
('SF18', '2014-01-01', 1) 7
('SF14', '2014-01-01', 0) 6
('SF11', '2014-01-01', 0) 2


### Part C


In [55]:
averaged.toDF().show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[SF04, 2014-01-04...|[15.3262499999999...|
|[SF04, 2014-01-04...|[16.1469444444444...|
|[SF04, 2014-01-04...|[18.2568571428571...|
|[SF04, 2014-01-04...|[17.5290476190476...|
|[SF04, 2014-01-04...|[17.7256249999999...|
|[SF04, 2014-01-04...|[17.1940425531914...|
|[SF04, 2014-01-04...|[17.8175000000000...|
|[SF04, 2014-01-04...|[17.2036111111111...|
|[SF04, 2014-01-04...|[15.5968421052631...|
|[SF04, 2014-01-04...|[13.464375, 0.774...|
|[SF04, 2014-01-04...|[11.2717391304347...|
|[SF04, 2014-01-04...|[12.0095454545454...|
|[SF04, 2014-01-04...|[11.9310000000000...|
|[SF04, 2014-01-05...|[15.1175, 4.22316...|
|[SF04, 2014-01-05...|[13.6367346938775...|
|[SF04, 2014-01-05...|[14.2456603773584...|
|[SF04, 2014-01-05...|[14.6088333333333...|
|[SF04, 2014-01-05...|[15.49, 2.9077333...|
|[SF04, 2014-01-05...|[16.4738333333333...|
|[SF04, 2014-01-05...|[16.639166

In [56]:
reduced.toDF().show()

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|[SF37, 2014-12-31...| 18|
|[SF07, 2014-12-31...|  1|
|[SF14, 2014-12-31...|  2|
|[SF11, 2014-12-31...|  6|
|[SF19, 2014-12-31...|  5|
|[SF34, 2014-12-31...|  2|
|[SF18, 2014-12-31...|  2|
|[SF36, 2014-12-31...|  1|
|[SF37, 2014-12-31...| 11|
|[SF07, 2014-12-31...|  1|
|[SF14, 2014-12-31...|  1|
|[SF19, 2014-12-31...|  2|
|[SF18, 2014-12-31...|  1|
|[SF34, 2014-12-31...|  2|
|[SF04, 2014-12-31...|  1|
|[SF37, 2014-12-31...|  7|
|[SF16, 2014-12-31...|  1|
|[SF19, 2014-12-31...|  1|
|[SF34, 2014-12-31...|  4|
|[SF18, 2014-12-31...|  1|
+--------------------+---+
only showing top 20 rows



In [57]:
# assume we have the wind averaged in RDD windaveraged
# and the incident counts in incidentsreduced
# and the keys for both are the same format


joined=averaged.join(reduced)

In [58]:
joined.toDF().show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[SF04, 2014-01-04...|[[17.817500000000...|
|[SF04, 2014-01-01...|[[14.978750000000...|
|[SF04, 2014-01-01...|[[13.95875, 1.596...|
|[SF04, 2014-01-02...|[[13.943333333333...|
|[SF04, 2014-01-02...|[[16.703958333333...|
|[SF04, 2014-01-02...|[[14.279574468085...|
|[SF04, 2014-01-03...|[[17.107708333333...|
|[SF04, 2014-01-03...|[[16.428780487804...|
|[SF04, 2014-01-02...|[[14.4496875, 1.2...|
|[SF04, 2014-01-01...|[[14.129090909090...|
|[SF04, 2014-01-02...|[[15.470833333333...|
|[SF04, 2014-01-04...| [[12.56, 0.158], 3]|
|[SF04, 2014-02-07...|[[8.7291666666666...|
|[SF04, 2014-02-01...|[[15.158115384615...|
|[SF04, 2014-02-02...|[[13.870624999999...|
|[SF04, 2014-02-02...|[[13.273333333333...|
|[SF04, 2014-02-03...|[[12.340208333333...|
|[SF04, 2014-02-03...|[[11.764166666666...|
|[SF04, 2014-02-03...|[[11.467291666666...|
|[SF04, 2014-02-03...|[[11.29000

In [59]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.stat import Statistics


In [61]:
#remap the data into a Vector of [t, w, i]
vecs = joined.map(lambda r: Vectors.dense([r[1][0][0],r[1][0][1],r[1][1]]))
print(Statistics.corr(vecs))


[[ 1.         -0.05716329 -0.01493063]
 [-0.05716329  1.         -0.03552069]
 [-0.01493063 -0.03552069  1.        ]]
